In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
import threading

## Group Assignment
### Team Number: 11
### Team Member Names: Bob Li, Lucas Zaharia, Charles Chen
### Team Strategy Chosen: RISKY

In [2]:
file_name="Tickers.csv"

# Read the File to a pd dataframe
ticker_list=pd.read_csv(file_name,header=None)


In [3]:
# Assign Start and End date for the analysis
start_date="2022-11-01"
end_date= "2022-12-30"

# Determine the minimum number of trading days required to be considered
apple =yf.Ticker("AAPL")
apple_hist = apple.history(start= start_date, end= end_date)
apple_hist.dropna(inplace=True)
min_data_len = len(apple_hist.index)

In [4]:
# Extract first trading day
apple_hist.reset_index(inplace=True)
first_trading_day = str(apple_hist["Date"][0])
first_trading_day = first_trading_day[0:10]

In [5]:
# add_month produces the trading volume in a given month and if the month has at least
# 20 trading days

# add_month: yf.Ticker Str Str -> [Monthly Volume, 0 or 1]
def add_month(ticker, start_here, end_here):
    # Create var called month_num representing if the month is eligible for calculation
    month_num=0
    
    # Get the history of month
    hist=ticker.history(start= start_here, end= end_here)
    
    # get a dataframe of volume
    company=hist["Volume"]
    
    # requires at least 20 trading days to be considered in the average
    if len(company) >= 20:
        # Calculate Sum
        month=company.sum()
        month_num+=1
    else:
        # Or do not add the month in
        month=0
    return [month, month_num]

# Calculate the Monthly Average of Volume of a stock
# Take in a ticker and returns monthly Average of that ticker
def monthly_avg(ticker):
    # Start variable total
    total=0
    
    # Start variable month_num representing the number of months
    month_num=0
    
    # Adding the volume for Jan-Sept
    for i in range(1,9):
        total+=add_month(ticker,f'2022-0{i}-01',f'2022-0{i+1}-01')[0]
        month_num+=add_month(ticker,f'2022-0{i}-01',f'2022-0{i+1}-01')[1]
    
    # Adding the volume for Sept-Oct
    total+=add_month(ticker,f'2022-09-01',f'2022-10-01')[0]
    month_num+=add_month(ticker,f'2022-09-01',f'2022-10-01')[1]
    
    total+=add_month(ticker,f'2022-10-01',f'2022-11-01')[0]
    month_num+=add_month(ticker,f'2022-10-01',f'2022-11-01')[1]
    
    # Calculating the monthly Average
    month_avg=total/month_num
    return month_avg
    

# Take in a dataframe of ticker name and an index (i)
# Function edits the indexed ticker name in the dataframe (remove it when it does not meet requirements)
def check_ticker_valid(i, ticker_list): 
    # Find indexed ticker name
    ticker=ticker_list.iloc[i,0]
    
    # Create the yfinance ticker
    company=yf.Ticker(ticker)
    
    # Use try except method to catch errors
    try:
        # Get the market of the ticker
        location=company.info['market']

        # Call monthly_avg to get the monthly average volume
        avg=monthly_avg(company)
        
        # calculate the length of data range
        company_hist = company.history(start= start_date, end= end_date)
        company_hist.dropna(inplace=True)
        lenn = len(company_hist.index)

        # To be eligible the stock must:
        #  - Be in US market
        #  - Have average monthly volume >=200000
        #  - Not an index
        #  - Has a certain number of trading days within time period (our requirement)
        if location != 'us_market' or avg < 200000 or lenn< min_data_len or "^" == ticker[0]:
            # Remove the Ticker from list if it does not meet requirements
            ticker_list.iloc[i,0]="N/A"
       
    except:
        # Remove the Ticker from list if it does not meet requirements
        ticker_list.iloc[i,0]="N/A"


In [6]:
# Create a list of threads
threads = []

# Start Threading in a loop
for i in range(0,len(ticker_list)):
    # Create a new thread
    t = threading.Thread(target=check_ticker_valid, args=[i, ticker_list])
    
    # Start thread
    t.start()
    
    # Add thread to thread list
    threads.append(t)

# Stop all threads in list
for thread in threads:
    thread.join()

- ACRV: Data doesn't exist for startDate = 1641013200, endDate = 1643691600
- ACRV: Data doesn't exist for startDate = 1641013200, endDate = 1643691600
- ACRV: Data doesn't exist for startDate = 1643691600, endDate = 1646110800
- ACRV: Data doesn't exist for startDate = 1643691600, endDate = 1646110800
- AUST: Data doesn't exist for startDate = 1641013200, endDate = 1643691600
- ACRV: Data doesn't exist for startDate = 1646110800, endDate = 1648785600
- AUST: Data doesn't exist for startDate = 1641013200, endDate = 1643691600
- ACRV: Data doesn't exist for startDate = 1646110800, endDate = 1648785600
- AUST: Data doesn't exist for startDate = 1643691600, endDate = 1646110800
- ACRV: Data doesn't exist for startDate = 1648785600, endDate = 1651377600
- AUST: Data doesn't exist for startDate = 1643691600, endDate = 1646110800
- ACRV: Data doesn't exist for startDate = 1648785600, endDate = 1651377600
- ACRV: Data doesn't exist for startDate = 1651377600, endDate = 1654056000
- AUST: Data

In [7]:
# NEW CODE
ticker_list = ticker_list.drop_duplicates()
# END NEW CODE

In [8]:
# Remove empty entries
ticker_list=ticker_list.replace('N/A',np.NaN)
ticker_list=ticker_list.dropna()

# Reset index
ticker_list.reset_index(drop=True, inplace=True)

In [9]:
# Create empty column called score
ticker_list['score']=''

# Calculates and insert score for each ticker in the dataframe
# Consumes Ticker list dataframe, index, start and end date
def eval_ticker(ticker_list,i,start_date, end_date):
    ticker_name=ticker_list.iloc[i,0]
    ticker=yf.Ticker(ticker_name)
    company=ticker.history(start= start_date, end= end_date)
    company=company["Close"]
    company=company.pct_change()
    company=company.dropna()
    standard_deviation=company.std()
    score=standard_deviation
    ticker_list['score'][i]=score

In [10]:
# Create a list of threads
threads = []

# Start Threading in a loop
for i in range(0,len(ticker_list)):
    # Create a new thread
    t = threading.Thread(target=eval_ticker, args=[ticker_list,i,start_date, end_date])
    
    # Start thread
    t.start()
    
    # Add thread to thread list
    threads.append(t)

# Stop all threads in list
for thread in threads:
    thread.join()


In [11]:
# Sort the dataframe by score in decreasing order
ticker_list.sort_values(by=['score'], ascending=False, inplace=True)

# Reset the index
ticker_list.reset_index(drop=True,inplace=True)


In [12]:
# Generates a dictionary where the key is the stock ticker and the associated value
# is a DataFrame containing daily close prices
# Parameters:
#  - ticker_lst (List): List containing stock tickers
# Returns:
#  - df_dict (Dictionary): Dictionary containing stock prices
def generate_stock_df_dict(ticker_lst):
    # Variables
    hist_interval = "1d"  
    df_dict = {}
    
    # Loop through each ticker in ticker_lst
    for ticker in ticker_lst:
        stock = yf.Ticker(ticker)
        
        # Get stock prices
        stock_hist = stock.history(start=start_date, interval=hist_interval)
        
        # Drop empty rows
        stock_hist.dropna(inplace=True)
        
        # Add DF with just Close column to dict
        df_dict[ticker] = stock_hist[["Close"]]
        
    # Return dict
    return df_dict

# Finds correlation in expected returns between two stocks when given a dictionary
# containing their stock prices.
# Parameters:
#  - ticker (String): Ticker of first stock
#  - second_ticker (String): Ticker of second stock
#  - df_dict (Dictionary): Dictionary containing stock prices
# Returns:
#  - returns_corr (Float): Correlation between expected returns
def find_corr(ticker, second_ticker, df_dict):
    # Variables
    ls = "_" + ticker
    rs = "_" + second_ticker
    
    # Join the price DataFrames for the two stocks
    prices = df_dict[ticker].join(df_dict[second_ticker], lsuffix=ls, rsuffix=rs)
    
    # Drop empty rows
    prices.dropna(inplace=True)
    
    # Get daily returns and drop first row
    daily_returns = prices.pct_change()
    daily_returns = daily_returns.iloc[1:]
    
    # Extract correlation
    returns_corr = daily_returns.corr().iloc[0, 1]
    
    # Return correlation
    return returns_corr

# Generates a list of portfolios, where each portfolio is composed of the 11 most
# correlated stocks to each stock in ticker_lst as well as the stock in ticker_lst
# Parameters:
#  - ticker_lst (List): List of tickers with length of at least 12
#  - df_dict (Dictionary): Dictionary containing stock prices
# Returns:
#  - portfolios (2D List): List of portfolios
def generate_portfolios(ticker_lst, df_dict):
    # Create empty list
    portfolios = []
    
    # Loop through each ticker in ticker_lst
    for ticker in ticker_lst:
        # Copy ticker_lst to avoid reference problems
        ticker_lst_copy = ticker_lst.copy()
        
        # Remove current ticker from copy
        ticker_lst_copy.remove(ticker)
        
        # Create a new DF where the index is ticker_lst_copy
        corr_df = pd.DataFrame(index=ticker_lst_copy)
        
        # Name the index Stock
        corr_df.index.name = "Stock"
        
        # Create a new column called Corr where each value is initially -2
        corr_df["Corr"] = -2
        
        # Loop through each ticker in ticker_lst_copy
        for second_ticker in ticker_lst_copy:
            # Find correlation between tickers in outer and inner for loops
            corr = find_corr(ticker, second_ticker, df_dict)
            
            # Place correlation value in appropriate location in DF
            corr_df.loc[second_ticker, "Corr"] = corr
        
        # Reset index
        corr_df.reset_index(inplace=True)
        
        # Sort DF values by Corr column from most correlated to least correlated
        corr_df.sort_values(by="Corr", ascending=False, inplace=True)
        
        # Create a new one element list containing ticker in outer for loop
        current_portfolio = [ticker]
        
        # Extract 11 most correlated stocks as a list
        additional_stocks = list(corr_df.head(11).Stock)
        
        # Extend the one element list to have the correlated stocks
        current_portfolio.extend(additional_stocks)
        
        # Append current portfolio to list of portfolios
        portfolios.append(current_portfolio)
    
    # Return list of portfolios
    return portfolios



In [13]:
# Keep 25 most risky stocks
ticker_list = ticker_list.head(25)

# Convert tickers into list
risky_tickers = list(ticker_list[0])

# Generate dictionary of stock prices
stock_price_dict = generate_stock_df_dict(risky_tickers)

# Generate 25 risky portfolios
portfolios = generate_portfolios(risky_tickers, stock_price_dict)

In [14]:
# Stores portfolio standard deviations
portfolio_stds = []

# Builds a portfolio with $500000 and adds the standard deviation of the portfolio to portfolio_stds
# Parameters: A list of 12 tickers
def buildportfolio(lst):
    portfolio = pd.DataFrame()
    
    #Determines how much of the portfolio the stock will be worth, based on its position in the list
    for x in range(len(lst)):
        if x <= 1:
            investment = 125000
        elif x == 2:
            investment = 62500
        else:
            investment = 62500/3
        
        #Gets closing price value from the dictionary defined earlier
        portfolio[lst[x]] = stock_price_dict[lst[x]]
        
        #Defines the number of shares based on the stock price from the first day
        try:
            num_shares = investment / portfolio.loc[first_trading_day, lst[x]]
        except:
            num_shares = investment / portfolio.loc["2022-11-01", lst[x]]
        
        #Multiplies closing value by number of shares 
        portfolio[lst[x]] = portfolio[lst[x]] * num_shares
        
    #Drops NaN values
    portfolio = portfolio.dropna()
    
    #Adds all stocks into a final column to track the portfolio value over time
    portfolio["portfolio value"] = portfolio.sum(axis = 1)
    
    #Adds the standard deviation of the portfolio to the portfolio_stds
    portfolio_stds.append(portfolio['portfolio value'].std())
    
#Applys buildportfolio to every list of tickers in portfolios
for n in portfolios:
    buildportfolio(n)

In [15]:
#Gets the list of tickers from portfolios based on the position of the largest standard deviation in portfolio_stds, since order is preserved
finalport_tickerslst = portfolios[portfolio_stds.index(max(portfolio_stds))]

finalport_pricelst = []

# Defines finalport_pricelst as the price of each ticker at the required date
for i in range(len(finalport_tickerslst)):
    finalport_pricelst.append(stock_price_dict[finalport_tickerslst[i]].loc["2022-11-25", "Close"])
    
finalport_shareslst = []

#Defines finalport_shareslst as the number of shares of each stock, based on finalport_pricelst and the position of the ticker in the finalport_tickerslst
for n in range(len(finalport_tickerslst)):
    value = -1
    if n <= 1:
        value = 125000
    elif n == 2:
        value = 62500
    else:
        value = 62500/3
    finalport_shareslst.append(value/finalport_pricelst[n])
    
#Creates a dictionary with the data above
finalport = {"Ticker" : finalport_tickerslst,
             "Price" : finalport_pricelst,
             "Shares" : finalport_shareslst}

#Creates the index for the portfolio
index_list = list(range(1, 13))

#Creates the dataframe from the data in finalport
Portfolio_Final = pd.DataFrame(finalport, index=index_list)

#Adds a column for Value using price * shares
Portfolio_Final["Value"] = Portfolio_Final.Price * Portfolio_Final.Shares

#Adds a column for Weight based oln the value of each stock
Portfolio_Final["Weight"] = Portfolio_Final.Value / 500000 * 100

Portfolio_Final

,Ticker,Price,Shares,Value,Weight
1,AUST,1.040000,120192.312101,125000.000000,25.000000
2,LOW,210.970001,592.501300,125000.000000,25.000000
3,SLB,50.830002,1229.588781,62500.000000,12.500000
4,OXY,70.279999,296.433320,20833.333333,4.166667
5,CMCSA,35.650002,584.385202,20833.333333,4.166667
6,AXP,154.149994,135.149751,20833.333333,4.166667
7,SHOP,36.790001,566.277054,20833.333333,4.166667
8,BMBL,22.049999,944.822406,20833.333333,4.166667
9,GOOG,97.599998,213.456287,20833.333333,4.166667
10,SQ,63.380001,328.705159,20833.333333,4.166667


In [16]:
print("Total Value:", Portfolio_Final.Value.sum())
print("Total Weight:", Portfolio_Final.Weight.sum())

Total Value: 499999.9999999999
Total Weight: 100.0


In [17]:
#Creates another dataframe using only the Ticker and Shares column from Portfolio_Final
Stocks_Final = Portfolio_Final[["Ticker", "Shares"]]

#Exports the dataframe to a csv file
Stocks_Final.to_csv('Stocks_Group_11.csv')

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Bob Li  
Lucas Zaharia  
Charles Chen